In [1]:
import Tiled_Classification_RF as TCRF
import os, tempfile
from osgeo import gdal, ogr, gdal_array # I/O image data
import numpy as np # math and array handling
import matplotlib.pyplot as plt # plot figures
from sklearn.ensemble import RandomForestClassifier # classifier
import pandas as pd # handling large data as table sheets
import geopandas as gpd # handling large data as shapefiles
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix  # calculating measures for accuracy assessment
import datetime
from xgboost import XGBClassifier
# Tell GDAL to throw Python exceptions, and register all drivers
gdal.UseExceptions()
gdal.AllRegister()
from GIStools.GIStools import preprocess_SfM_inputs
from GIStools.Stitch_Rasters import stitch_rasters
from GIStools.Grid_Creation import create_grid
from GIStools.Raster_Matching import pad_rasters_to_largest
from RF_input_parameters import TileClassifierParameters
import joblib

In [2]:
#-------------------Required User Defined Inputs-------------------#
params = TileClassifierParameters()

DEM_path = params.DEM_path
ortho_path = params.ortho_path
output_folder = params.output_folder
model_path = params.model_path
grid_ids_to_process  = params.grid_ids_to_process 
grid_path = params.grid_path
verbose = params.verbose
stitch = params.stitch

#--------------------Input Preparation-----------------------------#
in_dir = os.path.join(output_folder, 'RF_Tiled_Inputs')

results_folder = os.path.join(output_folder, 'RF_Results')
if not os.path.exists(results_folder):
    os.makedirs(results_folder)

classified_tile_folder = os.path.join(results_folder, 'Classified_Tiles')
if not os.path.exists(classified_tile_folder):
    os.makedirs(classified_tile_folder)
    
results_txt = os.path.join(output_folder, 'Results_Summary.txt') # directory, where the all meta results will be saved

In [3]:

#===========================Main Classification Loop===========================#
model = joblib.load(model_path)  # Load the saved model
for grid_id in grid_ids_to_process:
    print(f"\nProcessing grid {grid_id}")
    classification_image = os.path.join(classified_tile_folder, f"Classification_Tile_{grid_id}.tif")
    process_tile_path = os.path.join(in_dir, f"stacked_bands_tile_input_{grid_id}.tif") # path to the tile to be classified
    process_tile, process_tile_3Darray = TCRF.extract_image_data(process_tile_path, results_txt, log=True) # extract the training tile image data
    process_tile_2Darray = TCRF.flatten_raster_bands(process_tile_3Darray) # Convert NaNs to 0.0 and reshape the 3D array to 2D array
    class_prediction = TCRF.predict_classification(model, process_tile_2Darray, process_tile_3Darray) # predict the classification for each pixel using the trained model
    masked_prediction = TCRF.reshape_and_mask_prediction(class_prediction, process_tile_3Darray) # mask the prediction to only include bare earth and vegetation
    TCRF.save_classification_image(classification_image, process_tile, process_tile_3Darray, masked_prediction) # save the masked classification image

    del process_tile # close the image dataset


Processing grid 27
Extracting image data from: Y:\ATD\Drone Data Processing\GIS Processing\Random_Forest_BE_Classification\LM2\07092023\RF_Tiled_Inputs\stacked_bands_tile_input_27.tif
Image extent: 5459 x 9809 (row x col)
Number of Bands: 6
Reshaped from (5459, 9809, 6) to (53547331, 6)
No slicing was necessary!
Reshaped back to (5459, 9809)
Image saved to: Y:\ATD\Drone Data Processing\GIS Processing\Random_Forest_BE_Classification\LM2\07092023\RF_Results\Classified_Tiles\Classification_Tile_27.tif

Processing grid 29
Extracting image data from: Y:\ATD\Drone Data Processing\GIS Processing\Random_Forest_BE_Classification\LM2\07092023\RF_Tiled_Inputs\stacked_bands_tile_input_29.tif
Image extent: 5459 x 9809 (row x col)
Number of Bands: 6
Reshaped from (5459, 9809, 6) to (53547331, 6)
No slicing was necessary!
Reshaped back to (5459, 9809)
Image saved to: Y:\ATD\Drone Data Processing\GIS Processing\Random_Forest_BE_Classification\LM2\07092023\RF_Results\Classified_Tiles\Classification_Ti